## Settings

In [1]:
# CONTROLS
MODEL_PREFIX = "V24"
MODEL_NUMBER = MODEL_PREFIX[-2:]

TRAIN_SPLIT_RATIO = 0.2
BATCH_SIZE = 16
PREDICT_BATCH_SIZE = 512
DROPOUT = 0.3

RUN_ON_SAMPLE = False
EXCLUDE_NEUTRAL_CLASS = True
SENTIMENT_MAX_LR = 5e-4
SENTIMENT_MIN_LR = 5e-6
SENTIMENT_NUM_EPOCHS = [5, 5]
MAX_LR = 5e-3
MID_LR = 1e-4
MIN_LR = 1e-6
NUM_EPOCHS = [5, 5, 5]
NUM_FOLDS = 5

In [2]:
RESULTS_DIR = "../results/"
DATA_DIR = "../data/"

## Libraries

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.utils import class_weight

import pickle, os, sys, re, json, gc
from time import time, ctime
from pprint import pprint

import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv1D, Conv2D, LSTM, Embedding, Dense, concatenate, MaxPooling2D, Softmax, Flatten
from tensorflow.keras.layers import BatchNormalization, Dropout, Reshape, Activation, Bidirectional, TimeDistributed
from tensorflow.keras.layers import RepeatVector, Multiply, Layer, LeakyReLU, Subtract
from tensorflow.keras.activations import softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.callbacks import *
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
import tensorflow.keras.backend as K
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import save_model, load_model

import tokenizers, transformers
from transformers import *

%matplotlib inline

In [4]:
def jaccard(str1, str2):
    a = set(str1)
    b = set(str2)
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [5]:
seeded_value = 88888
pd.set_option('display.max_colwidth', None)
np.random.seed(seeded_value)

In [6]:
print(ctime(time()))

Mon Jun  1 01:20:26 2020


In [7]:
print([
    tf.__version__,
    transformers.__version__,
    tokenizers.__version__
])

['2.1.0', '2.8.0', '0.5.2']


<a href="https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth"  target="_blank"><h2 id="limiting_gpu_memory_growth" data-text="Limiting GPU memory growth" tabindex="0">Limiting GPU memory growth</h2></a>
<p>By default, TensorFlow maps nearly all of the GPU memory of all GPUs (subject to
<a href="https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars"><code translate="no" dir="ltr">CUDA_VISIBLE_DEVICES</code></a>) visible to the process. This is done to more efficiently use the relatively precious GPU memory resources on the devices by reducing memory fragmentation. To limit TensorFlow to a specific set of GPUs we use the <code translate="no" dir="ltr">tf.config.experimental.set_visible_devices</code> method.</p>

In [8]:
print(tf.config.experimental.list_logical_devices('CPU'))
print(tf.config.experimental.list_logical_devices('GPU'))
print(tf.config.experimental.list_physical_devices('CPU'))
print(tf.config.experimental.list_physical_devices('GPU'))

[LogicalDevice(name='/device:CPU:0', device_type='CPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

## Is it okay to exclude neutral text from models and force it later?

In [10]:
df2 = pd.read_csv(DATA_DIR+"train.csv", encoding="utf8")
# cases where neutral's text and selected text columns are the same
neutrals = df2.loc[df2.sentiment=="neutral"].copy()
neutrals = neutrals.reset_index(drop=True)
neutrals["text"] = neutrals["text"].astype(str)
neutrals["selected_text"] = neutrals["selected_text"].astype(str)
print("COVERAGE:", np.sum(np.where((neutrals.text == neutrals.selected_text), 1, 0))/neutrals.shape[0])
print("MEAN JACCARD:", np.mean([jaccard(str1=i, str2=j) for i,j in zip(neutrals.selected_text,neutrals.text)]))

neutrals.loc[neutrals.text != neutrals.selected_text].sample(5)

COVERAGE: 0.5334592552617378
MEAN JACCARD: 0.9808316180492903


,textID,text,selected_text,sentiment
4896,c68863089e,_tifullyTragic ... in London last night so give it a few hours & I`m sure a few will appear,in London last night so give it a few hours & I`m sure a few will appear,neutral
2211,edd2aceb1c,who sings `I Remember`? i alwaysss hear it on Radio 1 but never catch the artist,who sings `I Remember`? i alwaysss hear it on Radio 1 but never catch the artist,neutral
8286,ee53bf0e43,this time there is a theme and it is 'purple',this time there is a theme and it is 'purple',neutral
4240,2736a522fa,why isnt everyone with you?,why isnt everyone with you?,neutral
10069,8ba7a1720e,"and by the way it`s short stack, not sway sway sway sway baby is a song of theirs","and by the way it`s short stack, not sway sway sway sway baby is a song of theirs",neutral


## Import Data

In [11]:
df = pd.read_csv(DATA_DIR+"train.csv", encoding="utf8")

print(pd.concat((df.dtypes, df.isna().sum()), axis=1))
print(df.shape)

# Counts of various columns
print({i:df[i].nunique() for i in df.columns})
print(df.describe())
df.head(2)

                    0  1
textID         object  0
text           object  1
selected_text  object  1
sentiment      object  0
(27481, 4)
{'textID': 27481, 'text': 27480, 'selected_text': 22463, 'sentiment': 3}
            textID                                             text  \
count        27481                                            27480   
unique       27481                                            27480   
top     b5ddf13c3f  Missed the hello kitty  not enough time oh well   
freq             1                                                1   

       selected_text sentiment  
count          27480     27481  
unique         22463         3  
top             good   neutral  
freq             199     11118  


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative


In [12]:
test_df = pd.read_csv(DATA_DIR+"test.csv", encoding="utf8")
print(pd.concat((test_df.dtypes, test_df.isna().sum()), axis=1))
print(test_df.shape)

# Counts of various columns
print({i:test_df[i].nunique() for i in test_df.columns})
print(test_df.describe())
test_df.head(2)

                0  1
textID     object  0
text       object  0
sentiment  object  0
(3534, 3)
{'textID': 3534, 'text': 3534, 'sentiment': 3}
            textID  \
count         3534   
unique        3534   
top     aa221b6a7d   
freq             1   

                                                                                                                                       text  \
count                                                                                                                                  3534   
unique                                                                                                                                 3534   
top     The lab informs me the logic board is dead as well. It is farking expensive to replace. I don`t know what to do.  #MyPowerBookG4RIP   
freq                                                                                                                                      1   

       sentiment  
count       3

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,positive


In [13]:
df.loc[df['text'].astype('str').apply(lambda x : len(re.findall(pattern="ï¿½", string=x))>0)].head(2)

,textID,text,selected_text,sentiment
44,c77717b103,I love to! But I`m only available from 5pm. and where dear? Would love to help convert her vids.ï¿½,I love to!,positive
192,28dbada620,*phew* Will make a note in case anyone else runs into the same issueï¿½,*phew* Will make a note in case anyone else runs into the same issueï¿½,neutral


In [14]:
test_df.loc[test_df['text'].astype('str').apply(lambda x : len(re.findall(pattern="ï¿½", string=x))>0)].head(2)

,textID,text,sentiment
145,7223fdccc2,tikcets are only ï¿½91...each...BUT I SO WANT TO GO,positive
618,43ad351369,"AHHH - Whatchu talkinï¿½ baby? HAHAHA I canï¿½t believe youu:O heh, actually I can. Life is worth taking risks... http://tumblr.com/xs81qy54s",positive


In [15]:
df["set"], test_df["set"] = "train", "test"

### Create smaller sample for experimentation

In [16]:
if RUN_ON_SAMPLE:
    df = df.sample(500).reset_index(drop=True)
    test_df = test_df.sample(500).reset_index(drop=True)

#### Combine datasets for pretraining using sentiment labels

In [17]:
data = pd.concat((df[["text","set","sentiment"]],
                  test_df[["text","set","sentiment"]]), axis=0)
data["text"] = data["text"].astype(str)
data = data.sample(frac=1.0).reset_index(drop=True)
print(data.shape)

(31015, 3)


#### Sentiment count in combined data

In [18]:
data.groupby(["set","sentiment"])[["text"]].count()

text
set   sentiment       
test  negative    1001
      neutral     1430
      positive    1103
train negative    7781
      neutral    11118
      positive    8582

#### Tokenization

In [19]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer.save_pretrained(RESULTS_DIR+"tokenizers/roberta_tokenizer/")

('../results/tokenizers/roberta_tokenizer/vocab.json',
 '../results/tokenizers/roberta_tokenizer/merges.txt',
 '../results/tokenizers/roberta_tokenizer/special_tokens_map.json',
 '../results/tokenizers/roberta_tokenizer/added_tokens.json')

In [20]:
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=RESULTS_DIR+"tokenizers/roberta_tokenizer/vocab.json",
    merges_file=RESULTS_DIR+"tokenizers/roberta_tokenizer/merges.txt",
    add_prefix_space=True,
    lowercase=True)

In [21]:
with open(RESULTS_DIR+"tokenizers/roberta_tokenizer/special_tokens_map.json") as f:
    special_tokens = json.load(f)

tokenizer.add_special_tokens([i for i in special_tokens.values()])

0

#### Preprocessing for sentiment detection

In [22]:
sentiment_lookup = {"positive":2,"neutral":1,"negative":0}

In [23]:
def preprocess_sentiment(text_series=data.text.tolist(), sentiment_series=data.sentiment):

    X_tokens = tokenizer.encode_batch(text_series)

    X = [i.ids for i in X_tokens]
    MAX_SEQ_LEN = max([len(i) for i in X])
    X = pad_sequences(X, maxlen=MAX_SEQ_LEN, padding="post")

    X_att = [i.attention_mask for i in X_tokens]
    X_att = pad_sequences(X_att, maxlen=MAX_SEQ_LEN, padding="post")

    Y = sentiment_series.apply(lambda x: sentiment_lookup[x]).values

    VOCAB_SIZE = tokenizer.get_vocab_size()

    print({
        "X":X.shape,
        "X_att":X_att.shape,
        "Y":Y.shape,
        "VOCAB_SIZE":VOCAB_SIZE,
        "MAX_SEQ_LEN":MAX_SEQ_LEN
    })
    
    return X_tokens, X, X_att, Y, VOCAB_SIZE, MAX_SEQ_LEN

X_sent_tokens, X_sent, X_sent_att, Y_sent, VOCAB_SIZE, MAX_SEQ_LEN_SENT = preprocess_sentiment(**{
    "text_series" : data.text.tolist(),
    "sentiment_series" : data.sentiment
})

{'X': (31015, 100), 'X_att': (31015, 100), 'Y': (31015,), 'VOCAB_SIZE': 50265, 'MAX_SEQ_LEN': 100}


# Import data for span detection

In [24]:
df_span = pd.read_csv(DATA_DIR+"train.csv", encoding="utf8")

print(pd.concat((df_span.dtypes, df_span.isna().sum()), axis=1))
print(df_span.shape)

# Counts of various columns
print({i:df_span[i].nunique() for i in df_span.columns})
print(df_span.describe())
df_span.head(2)

                    0  1
textID         object  0
text           object  1
selected_text  object  1
sentiment      object  0
(27481, 4)
{'textID': 27481, 'text': 27480, 'selected_text': 22463, 'sentiment': 3}
            textID                                             text  \
count        27481                                            27480   
unique       27481                                            27480   
top     b5ddf13c3f  Missed the hello kitty  not enough time oh well   
freq             1                                                1   

       selected_text sentiment  
count          27480     27481  
unique         22463         3  
top             good   neutral  
freq             199     11118  


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative


In [25]:
test_df_span = pd.read_csv(DATA_DIR+"test.csv")
print(pd.concat((test_df_span.dtypes, test_df_span.isna().sum()), axis=1))
print(test_df_span.shape)

# Counts of various columns
print({i:test_df_span[i].nunique() for i in test_df_span.columns})
print(test_df_span.describe())
test_df_span.head(2)

                0  1
textID     object  0
text       object  0
sentiment  object  0
(3534, 3)
{'textID': 3534, 'text': 3534, 'sentiment': 3}
            textID  \
count         3534   
unique        3534   
top     aa221b6a7d   
freq             1   

                                                                                                                                       text  \
count                                                                                                                                  3534   
unique                                                                                                                                 3534   
top     The lab informs me the logic board is dead as well. It is farking expensive to replace. I don`t know what to do.  #MyPowerBookG4RIP   
freq                                                                                                                                      1   

       sentiment  
count       3

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,positive


#### Preprocessing for span detection

#### Cleaning for span detection

In [26]:
# For traceability
df_span["original_index"] = df_span.index
test_df_span["original_index"] = test_df_span.index

In [27]:
anomalous_idxs = [513, 1060, 5543, 8410, 13605, 24778, 25258] + [18, 27, 32, 39, 48, 49, 64, 66, 84, 92, 102, 116, 129, 132, 134, 149, 160, 166, 168, 189, 196, 197, 210, 223, 247, 251, 260, 285, 295, 296, 297, 309, 349, 362, 366, 382, 396, 398, 406, 410, 425, 458, 460, 482, 492, 500, 504, 517, 533, 537, 543, 569, 573, 581, 604, 605, 608, 634, 636, 637, 639, 642, 665, 670, 674, 678, 685, 702, 707, 709, 710, 727, 737, 746, 751, 757, 768, 778, 787, 794, 804, 809, 853, 854, 856, 863, 871, 872, 912, 931, 937, 942, 950, 956, 957, 963, 992, 993, 999, 1001, 1011, 1012, 1036, 1039, 1057, 1062, 1076, 1077, 1083, 1096, 1105, 1116, 1117, 1122, 1134, 1137, 1150, 1159, 1199, 1200, 1209, 1217, 1264, 1271, 1280, 1283, 1298, 1300, 1303, 1319, 1327, 1342, 1360, 1363, 1365, 1372, 1374, 1376, 1382, 1393, 1416, 1417, 1420, 1447, 1454, 1468, 1472, 1513, 1515, 1518, 1531, 1548, 1558, 1567, 1580, 1588, 1605, 1623, 1648, 1649, 1657, 1659, 1661, 1665, 1672, 1674, 1678, 1683, 1693, 1696, 1713, 1723, 1728, 1739, 1747, 1754, 1798, 1806, 1808, 1814, 1822, 1828, 1830, 1835, 1850, 1857, 1860, 1866, 1888, 1901, 1902, 1929, 1938, 1940, 1987, 1994, 2003, 2004, 2008, 2011, 2023, 2054, 2056, 2065, 2068, 2083, 2086, 2094, 2106, 2113, 2124, 2125, 2136, 2139, 2145, 2168, 2180, 2181, 2187, 2213, 2225, 2227, 2230, 2239, 2273, 2279, 2286, 2288, 2298, 2323, 2324, 2335, 2343, 2359, 2364, 2371, 2385, 2388, 2392, 2398, 2401, 2404, 2410, 2415, 2421, 2430, 2438, 2439, 2440, 2444, 2445, 2448, 2470, 2478, 2489, 2499, 2505, 2515, 2520, 2559, 2574, 2576, 2589, 2598, 2600, 2640, 2643, 2650, 2671, 2679, 2696, 2699, 2706, 2716, 2718, 2731, 2743, 2749, 2750, 2785, 2787, 2789, 2805, 2809, 2825, 2838, 2856, 2888, 2898, 2903, 2923, 2925, 2931, 2933, 2942, 2957, 2961, 2986, 2995, 3034, 3068, 3075, 3077, 3080, 3088, 3089, 3096, 3097, 3101, 3121, 3147, 3160, 3161, 3176, 3182, 3187, 3211, 3225, 3234, 3257, 3263, 3267, 3281, 3290, 3295, 3317, 3319, 3325, 3333, 3359, 3364, 3369, 3374, 3390, 3399, 3400, 3411, 3424, 3473, 3475, 3476, 3479, 3492, 3499, 3504, 3509, 3517, 3519, 3532, 3553, 3578, 3588, 3590, 3593, 3598, 3605, 3606, 3607, 3622, 3634, 3669, 3672, 3678, 3682, 3685, 3694, 3708, 3720, 3729, 3750, 3755, 3784, 3794, 3798, 3801, 3806, 3807, 3813, 3826, 3834, 3872, 3886, 3895, 3903, 3925, 3937, 3946, 3962, 3969, 3998, 4031, 4047, 4053, 4074, 4091, 4093, 4100, 4108, 4113, 4119, 4143, 4148, 4163, 4164, 4183, 4196, 4254, 4260, 4265, 4307, 4308, 4309, 4312, 4318, 4325, 4327, 4328, 4329, 4344, 4347, 4369, 4371, 4377, 4384, 4402, 4404, 4405, 4407, 4412, 4421, 4422, 4437, 4455, 4457, 4471, 4473, 4479, 4483, 4486, 4514, 4532, 4537, 4542, 4543, 4545, 4550, 4554, 4563, 4575, 4576, 4591, 4595, 4631, 4636, 4637, 4639, 4654, 4655, 4704, 4706, 4714, 4718, 4724, 4729, 4735, 4747, 4749, 4750, 4764, 4800, 4814, 4841, 4842, 4844, 4864, 4866, 4869, 4875, 4891, 4908, 4918, 4924, 4928, 4933, 4943, 4946, 4947, 4956, 4968, 4980, 4994, 5009, 5011, 5038, 5056, 5079, 5083, 5101, 5133, 5167, 5173, 5174, 5188, 5189, 5196, 5197, 5213, 5218, 5229, 5241, 5245, 5258, 5279, 5290, 5294, 5297, 5308, 5317, 5331, 5342, 5343, 5353, 5355, 5356, 5358, 5379, 5385, 5386, 5399, 5418, 5433, 5436, 5450, 5456, 5466, 5489, 5504, 5509, 5510, 5530, 5542, 5560, 5583, 5603, 5607, 5618, 5620, 5624, 5626, 5678, 5697, 5712, 5725, 5751, 5757, 5775, 5785, 5799, 5822, 5832, 5836, 5844, 5895, 5902, 5904, 5913, 5918, 5938, 5939, 5961, 5984, 5990, 6025, 6031, 6033, 6060, 6077, 6084, 6090, 6113, 6131, 6134, 6151, 6152, 6176, 6203, 6229, 6230, 6248, 6261, 6277, 6289, 6296, 6304, 6308, 6317, 6319, 6321, 6335, 6338, 6343, 6353, 6356, 6360, 6377, 6382, 6395, 6404, 6425, 6433, 6468, 6474, 6475, 6476, 6477, 6510, 6528, 6540, 6574, 6580, 6582, 6593, 6609, 6627, 6628, 6633, 6635, 6650, 6656, 6657, 6670, 6673, 6676, 6677, 6679, 6686, 6702, 6724, 6740, 6745, 6759, 6801, 6804, 6830, 6833, 6834, 6842, 6859, 6871, 6875, 6878, 6880, 6885, 6896, 6934, 6939, 6948, 6950, 6958, 6970, 6979, 6987, 7010, 7017, 7024, 7040, 7099, 7107, 7112, 7136, 7147, 7149, 7158, 7184, 7225, 7227, 7251, 7274, 7288, 7305, 7306, 7308, 7331, 7344, 7355, 7373, 7407, 7408, 7409, 7410, 7423, 7436, 7438, 7442, 7444, 7450, 7469, 7473, 7479, 7487, 7492, 7506, 7513, 7521, 7528, 7530, 7546, 7567, 7581, 7582, 7597, 7599, 7602, 7632, 7635, 7636, 7642, 7662, 7663, 7701, 7752, 7758, 7765, 7769, 7778, 7780, 7788, 7801, 7816, 7818, 7831, 7836, 7837, 7849, 7893, 7895, 7896, 7922, 7927, 7945, 7948, 7968, 7997, 8003, 8005, 8010, 8030, 8049, 8051, 8052, 8059, 8060, 8085, 8094, 8142, 8143, 8151, 8153, 8161, 8169, 8170, 8196, 8208, 8212, 8215, 8224, 8235, 8241, 8249, 8250, 8276, 8282, 8285, 8286, 8288, 8289, 8299, 8314, 8338, 8343, 8345, 8349, 8373, 8379, 8390, 8397, 8399, 8407, 8427, 8428, 8432, 8439, 8442, 8444, 8484, 8493, 8497, 8500, 8507, 8510, 8534, 8546, 8559, 8569, 8579, 8588, 8594, 8604, 8617, 8620, 8624, 8631, 8638, 8661, 8671, 8674, 8687, 8691, 8696, 8706, 8720, 8729, 8749, 8756, 8758, 8769, 8774, 8780, 8800, 8803, 8809, 8815, 8827, 8830, 8875, 8902, 8906, 8910, 8917, 8919, 8922, 8951, 8954, 8968, 8985, 8986, 8991, 8999, 9014, 9034, 9050, 9057, 9063, 9070, 9088, 9098, 9112, 9113, 9122, 9127, 9140, 9155, 9157, 9169, 9179, 9190, 9216, 9223, 9226, 9264, 9272, 9281, 9297, 9300, 9303, 9322, 9354, 9361, 9374, 9377, 9381, 9385, 9400, 9403, 9419, 9422, 9425, 9428, 9436, 9442, 9449, 9454, 9456, 9458, 9470, 9482, 9496, 9500, 9535, 9539, 9541, 9554, 9556, 9560, 9565, 9574, 9594, 9605, 9610, 9631, 9632, 9660, 9689, 9691, 9696, 9708, 9710, 9712, 9725, 9726, 9734, 9742, 9753, 9770, 9789, 9799, 9801, 9812, 9817, 9836, 9839, 9850, 9852, 9881, 9882, 9891, 9906, 9912, 9920, 9922, 9928, 9976, 9980, 9984, 10002, 10004, 10007, 10029, 10039, 10042, 10050, 10052, 10057, 10067, 10070, 10092, 10093, 10123, 10131, 10138, 10145, 10151, 10164, 10167, 10170, 10179, 10184, 10185, 10190, 10197, 10200, 10211, 10218, 10219, 10222, 10228, 10245, 10247, 10255, 10259, 10276, 10293, 10298, 10299, 10306, 10319, 10327, 10342, 10348, 10377, 10379, 10449, 10465, 10469, 10492, 10504, 10505, 10508, 10521, 10528, 10529, 10530, 10570, 10573, 10577, 10578, 10597, 10611, 10639, 10651, 10666, 10672, 10675, 10690, 10706, 10711, 10738, 10742, 10747, 10749, 10779, 10782, 10783, 10795, 10813, 10849, 10852, 10853, 10884, 10885, 10888, 10892, 10901, 10914, 10918, 10919, 10933, 10935, 10956, 10966, 10968, 10970, 10981, 10986, 10996, 10999, 11005, 11032, 11038, 11051, 11055, 11060, 11067, 11097, 11102, 11155, 11157, 11159, 11162, 11167, 11176, 11187, 11215, 11222, 11228, 11230, 11257, 11263, 11265, 11274, 11286, 11289, 11297, 11300, 11308, 11323, 11337, 11339, 11341, 11347, 11349, 11353, 11371, 11381, 11386, 11426, 11431, 11448, 11449, 11474, 11480, 11500, 11507, 11553, 11562, 11582, 11588, 11594, 11612, 11616, 11618, 11636, 11643, 11650, 11658, 11666, 11679, 11683, 11693, 11695, 11698, 11699, 11706, 11716, 11723, 11730, 11732, 11741, 11745, 11780, 11794, 11798, 11808, 11812, 11828, 11829, 11837, 11840, 11841, 11844, 11849, 11853, 11854, 11855, 11861, 11862, 11872, 11875, 11878, 11886, 11890, 11899, 11905, 11912, 11938, 11963, 11967, 11973, 11980, 11985, 11999, 12026, 12027, 12029, 12039, 12054, 12067, 12115, 12124, 12134, 12136, 12138, 12150, 12187, 12205, 12206, 12224, 12242, 12258, 12269, 12277, 12283, 12295, 12305, 12314, 12322, 12331, 12333, 12334, 12350, 12355, 12356, 12360, 12370, 12372, 12389, 12395, 12396, 12397, 12405, 12410, 12416, 12429, 12439, 12440, 12442, 12446, 12474, 12482, 12486, 12507, 12516, 12521, 12522, 12526, 12527, 12537, 12550, 12563, 12576, 12585, 12586, 12587, 12598, 12606, 12630, 12631, 12635, 12639, 12650, 12657, 12662, 12669, 12687, 12703, 12718, 12724, 12732, 12736, 12745, 12767, 12787, 12803, 12804, 12808, 12818, 12830, 12843, 12855, 12856, 12870, 12884, 12903, 12914, 12920, 12923, 12924, 12927, 12936, 12972, 12977, 12998, 13003, 13004, 13009, 13044, 13068, 13069, 13074, 13080, 13095, 13098, 13124, 13130, 13153, 13154, 13176, 13219, 13222, 13227, 13237, 13267, 13274, 13293, 13301, 13304, 13307, 13330, 13333, 13354, 13365, 13373, 13379, 13381, 13389, 13397, 13422, 13430, 13433, 13445, 13466, 13473, 13476, 13482, 13490, 13493, 13528, 13529, 13535, 13543, 13558, 13559, 13586, 13637, 13643, 13646, 13665, 13674, 13678, 13679, 13704, 13707, 13710, 13718, 13727, 13733, 13747, 13762, 13772, 13785, 13796, 13803, 13812, 13814, 13817, 13827, 13844, 13846, 13847, 13848, 13854, 13855, 13861, 13864, 13873, 13899, 13907, 13936, 13939, 13946, 13949, 13954, 13964, 13965, 13974, 13975, 13978, 13980, 14002, 14006, 14007, 14030, 14043, 14044, 14057, 14058, 14060, 14073, 14091, 14109, 14110, 14112, 14117, 14131, 14132, 14159, 14172, 14173, 14176, 14184, 14194, 14197, 14200, 14204, 14207, 14213, 14230, 14257, 14273, 14275, 14287, 14299, 14301, 14307, 14308, 14347, 14362, 14365, 14381, 14386, 14400, 14416, 14428, 14436, 14446, 14457, 14458, 14462, 14487, 14499, 14509, 14532, 14548, 14551, 14571, 14596, 14598, 14611, 14621, 14639, 14640, 14648, 14662, 14669, 14671, 14676, 14680, 14687, 14689, 14691, 14718, 14731, 14747, 14779, 14787, 14825, 14839, 14844, 14847, 14855, 14870, 14880, 14891, 14908, 14946, 14959, 14960, 14967, 14971, 14986, 15007, 15010, 15016, 15044, 15049, 15055, 15056, 15112, 15131, 15158, 15165, 15174, 15177, 15196, 15203, 15206, 15207, 15213, 15217, 15233, 15236, 15253, 15259, 15286, 15296, 15307, 15308, 15321, 15323, 15327, 15339, 15363, 15372, 15374, 15379, 15380, 15383, 15385, 15392, 15410, 15449, 15452, 15462, 15464, 15465, 15477, 15483, 15503, 15505, 15528, 15532, 15548, 15562, 15566, 15571, 15605, 15610, 15622, 15651, 15660, 15666, 15674, 15709, 15721, 15731, 15735, 15736, 15741, 15767, 15774, 15785, 15792, 15793, 15811, 15851, 15879, 15883, 15884, 15912, 15926, 15927, 15931, 15940, 15959, 15965, 15985, 15998, 16020, 16027, 16031, 16032, 16044, 16070, 16100, 16117, 16129, 16136, 16137, 16140, 16149, 16152, 16175, 16186, 16201, 16241, 16261, 16269, 16272, 16279, 16290, 16308, 16319, 16324, 16325, 16327, 16338, 16341, 16345, 16352, 16354, 16371, 16372, 16385, 16391, 16416, 16423, 16448, 16455, 16468, 16470, 16477, 16481, 16484, 16491, 16493, 16500, 16509, 16510, 16516, 16541, 16559, 16577, 16589, 16632, 16638, 16642, 16643, 16649, 16650, 16654, 16665, 16677, 16684, 16704, 16720, 16726, 16734, 16765, 16774, 16775, 16777, 16780, 16797, 16798, 16799, 16803, 16807, 16809, 16830, 16832, 16861, 16863, 16868, 16876, 16889, 16890, 16902, 16915, 16927, 16939, 16949, 16962, 16968, 17002, 17010, 17013, 17021, 17023, 17029, 17062, 17066, 17084, 17093, 17094, 17102, 17152, 17160, 17167, 17173, 17183, 17221, 17239, 17250, 17267, 17277, 17278, 17282, 17296, 17307, 17315, 17333, 17336, 17344, 17346, 17348, 17354, 17365, 17374, 17385, 17387, 17397, 17401, 17404, 17412, 17417, 17432, 17436, 17444, 17452, 17467, 17469, 17501, 17502, 17513, 17530, 17531, 17547, 17550, 17555, 17573, 17577, 17578, 17586, 17600, 17602, 17611, 17627, 17640, 17645, 17648, 17649, 17652, 17667, 17676, 17682, 17721, 17724, 17729, 17731, 17751, 17754, 17762, 17770, 17772, 17781, 17802, 17820, 17823, 17833, 17848, 17857, 17858, 17865, 17867, 17887, 17893, 17909, 17924, 17926, 17945, 17971, 17977, 17986, 17990, 18003, 18011, 18042, 18046, 18052, 18077, 18086, 18091, 18099, 18103, 18123, 18128, 18131, 18142, 18153, 18165, 18167, 18173, 18181, 18206, 18226, 18252, 18263, 18276, 18277, 18284, 18314, 18319, 18327, 18330, 18342, 18352, 18355, 18362, 18364, 18375, 18383, 18398, 18403, 18432, 18447, 18453, 18465, 18467, 18485, 18486, 18492, 18505, 18508, 18510, 18515, 18536, 18540, 18548, 18550, 18557, 18563, 18570, 18574, 18576, 18616, 18622, 18627, 18639, 18649, 18655, 18673, 18681, 18712, 18745, 18746, 18758, 18773, 18778, 18782, 18826, 18832, 18834, 18842, 18851, 18853, 18858, 18862, 18867, 18872, 18877, 18908, 18910, 18912, 18913, 18919, 18925, 18929, 18930, 18933, 18936, 18944, 18974, 18975, 18997, 19009, 19011, 19014, 19016, 19028, 19057, 19098, 19106, 19114, 19130, 19133, 19147, 19157, 19162, 19166, 19173, 19179, 19198, 19213, 19225, 19230, 19239, 19241, 19266, 19274, 19279, 19283, 19285, 19316, 19327, 19335, 19352, 19358, 19362, 19366, 19377, 19379, 19405, 19423, 19428, 19431, 19437, 19444, 19456, 19469, 19475, 19476, 19481, 19486, 19492, 19506, 19507, 19536, 19537, 19562, 19586, 19589, 19622, 19634, 19651, 19663, 19679, 19685, 19690, 19697, 19707, 19712, 19714, 19715, 19717, 19724, 19734, 19744, 19754, 19769, 19779, 19792, 19807, 19830, 19850, 19863, 19875, 19903, 19904, 19908, 19963, 19985, 19988, 20003, 20012, 20014, 20018, 20040, 20042, 20054, 20078, 20079, 20083, 20087, 20089, 20114, 20121, 20122, 20138, 20155, 20162, 20186, 20199, 20216, 20222, 20234, 20238, 20254, 20285, 20299, 20302, 20305, 20337, 20345, 20364, 20365, 20374, 20382, 20394, 20403, 20440, 20450, 20451, 20476, 20506, 20508, 20516, 20527, 20536, 20540, 20541, 20542, 20557, 20566, 20584, 20608, 20611, 20613, 20618, 20627, 20644, 20656, 20663, 20674, 20682, 20690, 20699, 20707, 20711, 20737, 20751, 20765, 20771, 20774, 20778, 20779, 20789, 20795, 20798, 20834, 20851, 20865, 20867, 20869, 20872, 20883, 20892, 20894, 20895, 20896, 20901, 20919, 20924, 20933, 20937, 20938, 20946, 20947, 20955, 20957, 20958, 20965, 20966, 20970, 20974, 20991, 21001, 21013, 21018, 21020, 21025, 21114, 21142, 21205, 21210, 21227, 21230, 21236, 21253, 21262, 21294, 21300, 21316, 21331, 21339, 21349, 21356, 21365, 21369, 21376, 21399, 21410, 21413, 21418, 21420, 21435, 21447, 21452, 21455, 21468, 21470, 21474, 21486, 21495, 21501, 21517, 21530, 21539, 21546, 21547, 21553, 21556, 21564, 21569, 21579, 21593, 21595, 21599, 21603, 21632, 21643, 21690, 21713, 21726, 21732, 21737, 21738, 21744, 21755, 21773, 21782, 21800, 21807, 21829, 21839, 21848, 21876, 21893, 21923, 21926, 21931, 21958, 21967, 21970, 21973, 21982, 22002, 22029, 22031, 22035, 22044, 22051, 22068, 22073, 22099, 22117, 22121, 22126, 22145, 22146, 22149, 22161, 22166, 22170, 22177, 22205, 22210, 22231, 22233, 22234, 22244, 22246, 22251, 22272, 22280, 22299, 22317, 22321, 22327, 22350, 22355, 22363, 22365, 22378, 22383, 22387, 22391, 22393, 22395, 22403, 22406, 22418, 22423, 22428, 22437, 22444, 22451, 22453, 22460, 22480, 22491, 22510, 22530, 22536, 22554, 22557, 22561, 22562, 22564, 22571, 22588, 22593, 22606, 22607, 22610, 22611, 22642, 22666, 22717, 22729, 22738, 22744, 22745, 22763, 22768, 22769, 22774, 22782, 22789, 22796, 22800, 22836, 22841, 22843, 22858, 22864, 22886, 22889, 22897, 22925, 22928, 22938, 22944, 22945, 22948, 22950, 22956, 22983, 22986, 22988, 22993, 22994, 22997, 23028, 23030, 23039, 23047, 23081, 23091, 23097, 23105, 23108, 23111, 23113, 23133, 23145, 23150, 23161, 23172, 23176, 23199, 23200, 23202, 23204, 23205, 23211, 23212, 23226, 23227, 23231, 23236, 23239, 23263, 23265, 23266, 23269, 23290, 23311, 23315, 23320, 23323, 23340, 23352, 23361, 23372, 23375, 23376, 23381, 23386, 23390, 23401, 23415, 23418, 23428, 23429, 23453, 23467, 23479, 23505, 23508, 23527, 23528, 23535, 23537, 23541, 23553, 23555, 23557, 23559, 23563, 23566, 23581, 23585, 23596, 23598, 23604, 23605, 23617, 23630, 23643, 23658, 23669, 23673, 23680, 23683, 23690, 23691, 23701, 23707, 23714, 23720, 23725, 23733, 23746, 23750, 23751, 23769, 23774, 23784, 23809, 23818, 23831, 23839, 23842, 23857, 23924, 23930, 23944, 23945, 23947, 23959, 23971, 23975, 23987, 23989, 24002, 24003, 24009, 24019, 24026, 24039, 24041, 24046, 24062, 24071, 24077, 24079, 24087, 24096, 24119, 24121, 24137, 24150, 24169, 24181, 24194, 24195, 24199, 24200, 24203, 24210, 24223, 24258, 24272, 24274, 24291, 24293, 24300, 24305, 24318, 24339, 24378, 24380, 24383, 24392, 24402, 24442, 24468, 24476, 24480, 24488, 24490, 24495, 24502, 24504, 24505, 24516, 24562, 24572, 24594, 24595, 24597, 24600, 24606, 24609, 24622, 24623, 24629, 24631, 24632, 24633, 24657, 24662, 24664, 24666, 24682, 24697, 24709, 24753, 24758, 24766, 24833, 24840, 24851, 24858, 24869, 24886, 24909, 24915, 24916, 24929, 24945, 24951, 24966, 24972, 24980, 24996, 25012, 25017, 25043, 25059, 25067, 25104, 25121, 25125, 25127, 25133, 25150, 25191, 25210, 25224, 25228, 25240, 25243, 25255, 25264, 25267, 25272, 25282, 25284, 25290, 25292, 25293, 25334, 25338, 25350, 25365, 25368, 25380, 25389, 25391, 25400, 25413, 25422, 25440, 25444, 25446, 25456, 25460, 25472, 25486, 25495, 25499, 25509, 25512, 25528, 25530, 25532, 25547, 25559, 25591, 25593, 25597, 25601, 25614, 25616, 25638, 25672, 25691, 25698, 25712, 25713, 25721, 25722, 25725, 25732, 25753, 25760, 25775, 25794, 25813, 25819, 25828, 25833, 25901, 25908, 25947, 25983, 25987, 25989, 25996, 26008, 26017, 26025, 26030, 26039, 26047, 26054, 26058, 26077, 26088, 26092, 26112, 26183, 26185, 26190, 26201, 26204, 26237, 26239, 26241, 26248, 26251, 26252, 26256, 26263, 26268, 26282, 26314, 26331, 26346, 26358, 26360, 26364, 26377, 26428, 26434, 26460, 26481, 26485, 26497, 26532, 26533, 26538, 26561, 26567, 26585, 26588, 26595, 26596, 26618, 26625, 26633, 26643, 26651, 26677, 26687, 26715, 26721, 26732, 26733, 26746, 26760, 26762, 26769, 26779, 26781, 26822, 26828, 26830, 26870, 26882, 26883, 26889, 26892, 26894, 26914, 26927, 26932, 26944, 26953, 26957, 26963, 26967, 26973, 26976, 26978, 26985, 27030, 27038, 27044, 27048, 27052, 27067, 27111, 27115, 27121, 27142, 27144, 27153, 27167, 27205, 27209, 27217, 27218, 27229, 27233, 27240, 27250, 27280, 27302, 27332, 27339, 27349, 27362, 27376, 27386, 27396, 27401, 27413, 27426, 27429, 27456, 27470, 27474, 27476, 27477, 27480]

In [28]:
print(df_span.shape)
df_span = df_span[(~df_span.index.isin(anomalous_idxs))]
print(df_span.shape)
df_span = df_span[(~df_span.text.isna())]
df_span = df_span.reset_index(drop=True)
df_span = df_span.copy()
print(df_span.shape)

(27481, 5)
(24857, 5)
(24856, 5)


In [29]:
df_span["sentiment_code"] = df_span["sentiment"].astype("category")
X_sentiments = df_span["sentiment_code"].cat.codes.values

test_df_span["sentiment_code"] = test_df_span["sentiment"].astype("category")
X_sentiments_test = test_df_span["sentiment_code"].cat.codes.values

In [30]:
df_span["text"] = df_span["text"].astype(str)
df_span["selected_text"] = df_span["selected_text"].astype(str)
test_df_span["text"] = test_df_span["text"].astype(str)

In [31]:
df_span["text_mod"] = "<s> " + df_span.text.str.strip() + " </s> " + df_span.sentiment + " </s>"
test_df_span["text_mod"] = "<s> " + test_df_span.text.str.strip() + " </s> " + test_df_span.sentiment + " </s>"

#### Exclusions for span detection

In [32]:
if EXCLUDE_NEUTRAL_CLASS:
    df_span = df_span.loc[df_span.sentiment!="neutral"].copy()
    df_span = df_span.reset_index(drop=True)
    print("EXCLUDE_NEUTRAL_CLASS:", df_span.shape)

if RUN_ON_SAMPLE:
    df_span = df_span.sample(1000).copy()
    df_span = df_span.reset_index(drop=True)
    print("Train RUN_ON_SAMPLE", df_span.shape)
    test_df_span = test_df_span.sample(1000).copy()
    test_df_span = test_df_span.reset_index(drop=True)
    print("Test  RUN_ON_SAMPLE", test_df_span.shape)

EXCLUDE_NEUTRAL_CLASS: (14017, 7)


#### Tokenization for span detection

In [33]:
X_span_tokens = tokenizer.encode_batch(df_span.text_mod.tolist())
Y_span_tokens = tokenizer.encode_batch(df_span.selected_text.tolist())
X_span_tokens_test = tokenizer.encode_batch(test_df_span.text_mod.tolist())

In [34]:
X_span = [i.ids for i in X_span_tokens]
Y_span = [i.ids for i in Y_span_tokens]
X_span_test = [i.ids for i in X_span_tokens_test]

In [35]:
X_span_att = [i.attention_mask for i in X_span_tokens]
Y_span_att = [i.attention_mask for i in Y_span_tokens]
X_span_att_test = [i.attention_mask for i in X_span_tokens_test]

In [36]:
MAX_SEQ_LEN_SPAN = max([len(i) for i in X_span])

In [37]:
def get_extremities(l_string, s_string, print_it=False):
    len_l = len(l_string)
    len_s = len(s_string)
    
    for i in range(len_l - len_s + 1):
        if (i + len_s) <= len_l:
            substring = l_string[i:i+len_s]
            if substring == s_string:
                if print_it:
                    print(l_string)
                    print(substring)
                    print(i, i+len_s, substring)
                
                start_vector, end_vector = [0] * len_l, [0] * len_l
                start_vector[i], end_vector[i+len_s-1] = 1, 1
                
                return (start_vector, end_vector)

In [38]:
Y_span_starts, Y_span_stops = [], []
anomaly_idx, counter = [], 0
for num, (i,j) in enumerate(zip(X_span_tokens, Y_span_tokens)):
    x,y = i.ids, j.ids
    try:
        s,e = get_extremities(x, y)
    except TypeError as t:
        counter += 1
        anomaly_idx.append(num)
    Y_span_starts.append(s)
    Y_span_stops.append(e)
print(num + 1, "\t: #Processed")

print(counter,"\t: # of Anomalies")

14017 	: #Processed
0 	: # of Anomalies


In [39]:
if counter:
    print(anomaly_idx, sep=",")
    print(df_span["original_index"].iloc[anomaly_idx].tolist())
    df_span.iloc[anomaly_idx].tail(10)

In [40]:
check_idx = 156
print(df_span.text[check_idx])
print(df_span.selected_text[check_idx])
print([[i,j,k,l] for i,j,k,l in zip(X_span_tokens[check_idx].tokens,
                                    X_span_tokens[check_idx].ids,
                                    Y_span_starts[check_idx],
                                    Y_span_stops[check_idx])])
print([[i,j] for i,j in zip(Y_span_tokens[check_idx].ids,
                            Y_span_tokens[check_idx].tokens)])

mcfly gig last nightt omg it was amazin didnt sit down through the whole thing  mcfly did you see me and ma best mate we were in tutus
it was amazin
[['<s>', 0, 0, 0], ['Ġmc', 44355, 0, 0], ['fly', 19252, 0, 0], ['Ġgig', 10196, 0, 0], ['Ġlast', 94, 0, 0], ['Ġnight', 363, 0, 0], ['t', 90, 0, 0], ['Ġo', 1021, 0, 0], ['mg', 22984, 0, 0], ['Ġit', 24, 1, 0], ['Ġwas', 21, 0, 0], ['Ġamaz', 42402, 0, 0], ['in', 179, 0, 1], ['Ġdidnt', 46405, 0, 0], ['Ġsit', 2662, 0, 0], ['Ġdown', 159, 0, 0], ['Ġthrough', 149, 0, 0], ['Ġthe', 5, 0, 0], ['Ġwhole', 1086, 0, 0], ['Ġthing', 631, 0, 0], ['Ġ', 1437, 0, 0], ['Ġmc', 44355, 0, 0], ['fly', 19252, 0, 0], ['Ġdid', 222, 0, 0], ['Ġyou', 47, 0, 0], ['Ġsee', 192, 0, 0], ['Ġme', 162, 0, 0], ['Ġand', 8, 0, 0], ['Ġma', 9131, 0, 0], ['Ġbest', 275, 0, 0], ['Ġmate', 12563, 0, 0], ['Ġwe', 52, 0, 0], ['Ġwere', 58, 0, 0], ['Ġin', 11, 0, 0], ['Ġtut', 15511, 0, 0], ['us', 687, 0, 0], ['Ġ', 1437, 0, 0], ['</s>', 2, 0, 0], ['Ġpositive', 1313, 0, 0], ['Ġ', 1437, 0, 0], ['</s

#### Padding for span detection

In [41]:
X_span = pad_sequences(X_span, maxlen=MAX_SEQ_LEN_SPAN, padding="post")
X_span_att = pad_sequences(X_span_att, maxlen=MAX_SEQ_LEN_SPAN, padding="post")
Y_span = pad_sequences(Y_span, maxlen=MAX_SEQ_LEN_SPAN, padding="post")

Y_span_starts = pad_sequences(Y_span_starts, maxlen=MAX_SEQ_LEN_SPAN, padding="post").argmax(axis=1)
Y_span_stops = pad_sequences(Y_span_stops, maxlen=MAX_SEQ_LEN_SPAN, padding="post").argmax(axis=1)

X_span_test = pad_sequences(X_span_test, maxlen=MAX_SEQ_LEN_SPAN, padding="post")
X_span_att_test = pad_sequences(X_span_att_test, maxlen=MAX_SEQ_LEN_SPAN, padding="post")

In [42]:
pprint({
    "X_span" : X_span.shape,
    "X_span_att" : X_span_att.shape,
    "Y_span" : Y_span.shape,
    "Y_span_starts" : Y_span_starts.shape,
    "Y_span_stops" : Y_span_stops.shape,
    "X_span_test" : X_span_test.shape,
    "X_span_att_test" : X_span_att_test.shape,
    "VOCAB_SIZE":VOCAB_SIZE,
    "MAX_SEQ_LEN_SPAN":MAX_SEQ_LEN_SPAN
})

{'MAX_SEQ_LEN_SPAN': 76,
 'VOCAB_SIZE': 50265,
 'X_span': (14017, 76),
 'X_span_att': (14017, 76),
 'X_span_att_test': (3534, 76),
 'X_span_test': (3534, 76),
 'Y_span': (14017, 76),
 'Y_span_starts': (14017,),
 'Y_span_stops': (14017,)}


#### Cross validation for span detection

In [43]:
keep_flag = np.isin(Y_span_stops,
                    np.unique(Y_span_stops,
                              return_counts=True)[0][np.unique(Y_span_stops,
                                                               return_counts=True)[1]>1])

In [44]:
print("\n",
     X_span.shape, "\t: X ", "\n",
     X_span_att.shape, "\t: X_att ", "\n",
     Y_span.shape, "\t: Y ", "\n",
     Y_span_starts.shape, "\t: Y_starts ", "\n",
     Y_span_stops.shape, "\t: Y_stops ", "\n",
     X_span_test.shape, "\t: X_test ", "\n",
     X_span_att_test.shape, "\t: X_att_test ", "\n"
)


 (14017, 76) 	: X  
 (14017, 76) 	: X_att  
 (14017, 76) 	: Y  
 (14017,) 	: Y_starts  
 (14017,) 	: Y_stops  
 (3534, 76) 	: X_test  
 (3534, 76) 	: X_att_test  



In [45]:
X_span = X_span[keep_flag]
X_span_att = X_span_att[keep_flag]
Y_span = Y_span[keep_flag]
Y_span_starts = Y_span_starts[keep_flag]
Y_span_stops = Y_span_stops[keep_flag]
X_span_test = X_span_test
X_span_att_test = X_span_att_test

In [46]:
print("\n",
     X_span.shape, "\t: X ", "\n",
     X_span_att.shape, "\t: X_att ", "\n",
     Y_span.shape, "\t: Y ", "\n",
     Y_span_starts.shape, "\t: Y_starts ", "\n",
     Y_span_stops.shape, "\t: Y_stops ", "\n",
     X_span_test.shape, "\t: X_test ", "\n",
     X_span_att_test.shape, "\t: X_att_test ", "\n"
)


 (14015, 76) 	: X  
 (14015, 76) 	: X_att  
 (14015, 76) 	: Y  
 (14015,) 	: Y_starts  
 (14015,) 	: Y_stops  
 (3534, 76) 	: X_test  
 (3534, 76) 	: X_att_test  



In [47]:
#sss = StratifiedKFold(n_splits=NUM_FOLDS,shuffle=True,random_state=seeded_value)
sss = KFold(n_splits=NUM_FOLDS,shuffle=True,random_state=seeded_value)

In [48]:
Y_span_words = [tokenizer.decode(i) for i in Y_span]

In [49]:
MAX_SEQ_LEN = max(MAX_SEQ_LEN_SENT, MAX_SEQ_LEN_SPAN)

In [50]:
X_span = pad_sequences(X_span, maxlen=MAX_SEQ_LEN, padding="post")
X_span_att = pad_sequences(X_span_att, maxlen=MAX_SEQ_LEN, padding="post")
Y_span = pad_sequences(Y_span, maxlen=MAX_SEQ_LEN, padding="post")

X_span_test = pad_sequences(X_span_test, maxlen=MAX_SEQ_LEN, padding="post")
X_span_att_test = pad_sequences(X_span_att_test, maxlen=MAX_SEQ_LEN, padding="post")

X_sent = pad_sequences(X_sent, maxlen=MAX_SEQ_LEN, padding="post")
X_sent_att = pad_sequences(X_sent_att, maxlen=MAX_SEQ_LEN, padding="post")

pprint({
    "X_span" : X_span.shape,
    "X_span_att" : X_span_att.shape,
    "Y_span" : Y_span.shape,
    
    "X_span_test" : X_span_test.shape,
    "X_span_att_test" : X_span_att_test.shape,
    
    "X_sent" : X_sent.shape,
    "X_sent_att" : X_sent_att.shape,
})

{'X_sent': (31015, 100),
 'X_sent_att': (31015, 100),
 'X_span': (14015, 100),
 'X_span_att': (14015, 100),
 'X_span_att_test': (3534, 100),
 'X_span_test': (3534, 100),
 'Y_span': (14015, 100)}


#### Model Specifications

In [51]:
def build_model():
    input_sequences = Input((MAX_SEQ_LEN), dtype=tf.int32, name="words")
    input_att_flags = Input((MAX_SEQ_LEN), dtype=tf.int32, name="att_flags")
    input_token_ids = Input((MAX_SEQ_LEN), dtype=tf.int32, name="token_ids")
    
    config = RobertaConfig.from_pretrained('roberta-base')
    roberta_model = TFRobertaModel.from_pretrained('roberta-base', config=config)
    x = roberta_model(inputs=input_sequences, attention_mask=input_att_flags, token_type_ids=input_token_ids)
    
    x1 = tf.keras.layers.Dropout(DROPOUT)(x[0])
    x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    output_starts_0 = tf.keras.layers.Activation('softmax', name="starts_0")(x1)
    
    x2 = tf.keras.layers.Dropout(DROPOUT)(x[0]) 
    x2 = tf.keras.layers.Conv1D(768, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    output_stops_0 = tf.keras.layers.Activation('softmax', name="stops_0")(x2)
    
    x3 = tf.keras.layers.Dropout(DROPOUT)(x[0]) 
    x3 = tf.keras.layers.Conv1D(768, 2,padding='same')(x3)
    x3 = tf.keras.layers.LeakyReLU()(x3)
    x3 = tf.keras.layers.Dense(1)(x3)
    x3 = tf.keras.layers.Flatten()(x3)
    x3 = tf.keras.layers.Dense(3)(x3)
    output_sentiment = tf.keras.layers.Activation('softmax', name="output_sentiments")(x3)
    
    sentiment_model = Model([input_att_flags, input_sequences, input_token_ids], [output_sentiment])
    
    output_subtract = tf.keras.layers.Subtract()([output_starts_0, output_stops_0])
    output_flat = concatenate([output_starts_0, output_stops_0, output_subtract, output_sentiment])
    output_starts_1 = Dense(MAX_SEQ_LEN, activation='softmax', name="starts_1")(output_flat)
    output_stops_1 = Dense(MAX_SEQ_LEN, activation='softmax', name="stops_1")(output_flat)
    
    span_detection_model = Model([input_att_flags, input_sequences, input_token_ids],
                                 [output_starts_0, output_stops_0, output_starts_1, output_stops_1])
    
    return sentiment_model, span_detection_model

In [52]:
sentiment_model, span_detection_model = build_model()

In [53]:
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words (InputLayer)              [(None, 100)]        0                                            
__________________________________________________________________________________________________
att_flags (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
token_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 100, 768), ( 124645632   words[0][0]                      
______________________________________________________________________________________________

In [54]:
span_detection_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words (InputLayer)              [(None, 100)]        0                                            
__________________________________________________________________________________________________
att_flags (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
token_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 100, 768), ( 124645632   words[0][0]                      
____________________________________________________________________________________________

### Sentiment Model Fit

In [55]:
sentiment_mcp = ModelCheckpoint(filepath=RESULTS_DIR+"Sentiment_"+MODEL_PREFIX+"BestCheckpoint.h5", monitor='val_loss',
                                verbose=0, save_best_only=False, save_weights_only=True, mode='auto', save_freq='epoch')

sentiment_csvl = CSVLogger(filename=RESULTS_DIR+"Sentiment_"+MODEL_PREFIX+"_LossLogs.csv",
                           separator=",", append=True)

In [56]:
tr_index, va_index, _, _ = train_test_split(np.arange(X_sent.shape[0]),
                                            np.arange(Y_sent.shape[0]),
                                            test_size=TRAIN_SPLIT_RATIO,
                                            random_state=seeded_value,
                                            shuffle=True)

In [57]:
p,c = np.unique(Y_sent, return_counts=True)
cw = class_weight.compute_class_weight('balanced', np.unique(Y_sent), Y_sent)

In [58]:
sentiment_model.layers[3].trainable = False
adam = Adam(learning_rate=SENTIMENT_MAX_LR)
sentiment_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        optimizer=adam,
                        metrics=['accuracy'])

In [59]:
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words (InputLayer)              [(None, 100)]        0                                            
__________________________________________________________________________________________________
att_flags (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
token_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 100, 768), ( 124645632   words[0][0]                      
______________________________________________________________________________________________

In [60]:
sent_history = sentiment_model.fit(x={"att_flags":X_sent_att[tr_index],
                                      "words":X_sent[tr_index],
                                      "token_ids":np.zeros_like(X_sent_att[tr_index])},
                                   y={"output_sentiments":Y_sent[tr_index]},
                                   shuffle=True,
                                   batch_size=BATCH_SIZE,
                                   epochs=SENTIMENT_NUM_EPOCHS[0],
                                   validation_data=({"att_flags":X_sent_att[va_index],
                                                     "words":X_sent[va_index],
                                                     "token_ids":np.zeros_like(X_sent_att[va_index])},
                                                    {"output_sentiments":Y_sent[va_index]}),
                                   verbose=1,
                                   class_weight=cw,
                                   callbacks=[sentiment_mcp, sentiment_csvl])

Train on 24812 samples, validate on 6203 samples
Epoch 1/5
24812/24812 [==============================] - 293s 12ms/sample - loss: 1.0098 - accuracy: 0.4847 - val_loss: 0.8771 - val_accuracy: 0.5659
Epoch 2/5
24812/24812 [==============================] - 274s 11ms/sample - loss: 0.8975 - accuracy: 0.5652 - val_loss: 0.8576 - val_accuracy: 0.5771
Epoch 3/5
24812/24812 [==============================] - 274s 11ms/sample - loss: 0.8614 - accuracy: 0.5931 - val_loss: 0.7767 - val_accuracy: 0.6497
Epoch 4/5
24812/24812 [==============================] - 274s 11ms/sample - loss: 0.8396 - accuracy: 0.6121 - val_loss: 0.7656 - val_accuracy: 0.6618
Epoch 5/5
24812/24812 [==============================] - 276s 11ms/sample - loss: 0.8258 - accuracy: 0.6242 - val_loss: 0.7777 - val_accuracy: 0.6565


In [61]:
sentiment_model.layers[3].trainable = True
adam = Adam(learning_rate=SENTIMENT_MIN_LR)
sentiment_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        optimizer=adam,
                        metrics=['accuracy'])

In [62]:
sent_history_finetuned = sentiment_model.fit(x={"att_flags":X_sent_att[tr_index],
                                                "words":X_sent[tr_index],
                                                "token_ids":np.zeros_like(X_sent_att[tr_index])},
                                             y={"output_sentiments":Y_sent[tr_index]},
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             epochs=SENTIMENT_NUM_EPOCHS[1],
                                             validation_data=({"att_flags":X_sent_att[va_index],
                                                               "words":X_sent[va_index],
                                                               "token_ids":np.zeros_like(X_sent_att[va_index])},
                                                              {"output_sentiments":Y_sent[va_index]}),
                                             verbose=1,
                                             class_weight=cw,
                                             callbacks=[sentiment_mcp, sentiment_csvl])

Train on 24812 samples, validate on 6203 samples
Epoch 1/5
24812/24812 [==============================] - 653s 26ms/sample - loss: 0.6856 - accuracy: 0.7135 - val_loss: 0.5921 - val_accuracy: 0.7595
Epoch 2/5
24812/24812 [==============================] - 638s 26ms/sample - loss: 0.5885 - accuracy: 0.7563 - val_loss: 0.5586 - val_accuracy: 0.7706
Epoch 3/5
24812/24812 [==============================] - 637s 26ms/sample - loss: 0.5327 - accuracy: 0.7863 - val_loss: 0.5681 - val_accuracy: 0.7712
Epoch 4/5
24812/24812 [==============================] - 635s 26ms/sample - loss: 0.4721 - accuracy: 0.8078 - val_loss: 0.6016 - val_accuracy: 0.7787
Epoch 5/5
24812/24812 [==============================] - 635s 26ms/sample - loss: 0.4262 - accuracy: 0.8306 - val_loss: 0.5643 - val_accuracy: 0.7733


#### Validate sentiment model

In [63]:
sample_text = data.text.sample(5).tolist()

encoded_repr = tokenizer.encode_batch(sample_text)

sample_text_ids = pad_sequences([i.ids for i in encoded_repr],
                                maxlen=MAX_SEQ_LEN,
                                padding="post")
sample_text_att = pad_sequences([i.attention_mask for i in encoded_repr],
                                maxlen=MAX_SEQ_LEN,
                                padding="post")
pred = sentiment_model.predict({"att_flags":sample_text_att,
                                "words":sample_text_ids,
                                "token_ids":np.zeros_like(sample_text_att)})

pprint({
    num:{
        "text":i,
        "predicted_sentiment":[k for k,v in sentiment_lookup.items() if v==j][0]
    } for num,(i,j) in enumerate(zip(sample_text, pred.argmax(axis=1)))
})

{0: {'predicted_sentiment': 'negative',
     'text': 'Going out to Miranda shopping centre to spend time with the '
             'family, before going away for 2 weeks to Malaysia. Gonna miss '
             'them!'},
 1: {'predicted_sentiment': 'neutral',
     'text': '  Now I need to find the Keynote one! At least I know where to '
             'go! #ScreenCastsOnline'},
 2: {'predicted_sentiment': 'positive',
     'text': 'relaxing night at home with best people'},
 3: {'predicted_sentiment': 'negative',
     'text': 'Watching Ace of Cakes: LOST edition omfgggg'},
 4: {'predicted_sentiment': 'negative',
     'text': 'doneeee wheeee hahaaaaaaaa so tired and sleepy  peter u suck not '
             'coming to my bday!'}}


In [64]:
def infer_sentiment(x):
    encoded_repr = tokenizer.encode_batch(x.tolist())

    sample_text_ids = pad_sequences([i.ids for i in encoded_repr],
                                    maxlen=MAX_SEQ_LEN,
                                    padding="post")
    sample_text_att = pad_sequences([i.attention_mask for i in encoded_repr],
                                    maxlen=MAX_SEQ_LEN,
                                    padding="post")
    pred = sentiment_model.predict({"att_flags":sample_text_att,
                                    "words":sample_text_ids,
                                    "token_ids":np.zeros_like(sample_text_att)},
                                  batch_size=PREDICT_BATCH_SIZE)
    
    res = pd.DataFrame({"predicted_sentiment":pred.argmax(axis=1)})
    
    return res.predicted_sentiment.apply(lambda x:[k for k,v in sentiment_lookup.items() if v==x][0])

In [65]:
%%time
data["predicted_sentiment"] = infer_sentiment(x=data.text)

Wall time: 3min 15s


In [66]:
print(classification_report(y_true=data.sentiment[data.index.isin(tr_index)],
                            y_pred=data.predicted_sentiment[data.index.isin(tr_index)]))

              precision    recall  f1-score   support

    negative       0.90      0.90      0.90      7047
     neutral       0.87      0.88      0.87     10041
    positive       0.92      0.90      0.91      7724

    accuracy                           0.89     24812
   macro avg       0.90      0.89      0.90     24812
weighted avg       0.89      0.89      0.89     24812



In [67]:
print(classification_report(y_true=data.sentiment[data.index.isin(va_index)],
                            y_pred=data.predicted_sentiment[data.index.isin(va_index)]))

              precision    recall  f1-score   support

    negative       0.77      0.78      0.78      1735
     neutral       0.73      0.74      0.74      2507
    positive       0.83      0.81      0.82      1961

    accuracy                           0.77      6203
   macro avg       0.78      0.78      0.78      6203
weighted avg       0.77      0.77      0.77      6203



In [68]:
confusion_matrix(y_true=data.sentiment,
                 y_pred=data.predicted_sentiment,
                 labels=['positive', 'neutral', 'negative'])

array([[ 8557,  1023,   105],
       [  848, 10698,  1002],
       [   85,  1003,  7694]], dtype=int64)

In [69]:
confusion_matrix(y_true=data.sentiment,
                 y_pred=data.predicted_sentiment,
                 labels=['positive', 'neutral', 'negative'],
                 normalize="all")

array([[0.27589876, 0.03298404, 0.00338546],
       [0.02734161, 0.34492987, 0.03230695],
       [0.00274061, 0.03233919, 0.24807351]])

In [70]:
data["set2"] = np.where(data.index.isin(tr_index), "train", "valid")
data.groupby("set2").apply(lambda x : accuracy_score(y_true=x.sentiment, y_pred=x.predicted_sentiment))

set2
train    0.892794
valid    0.773335
dtype: float64

In [71]:
data.groupby("set").apply(lambda x : accuracy_score(y_true=x.sentiment, y_pred=x.predicted_sentiment))

set
test     0.874363
train    0.868200
dtype: float64

In [72]:
data.groupby(["set", "set2"])["sentiment"].count()

set    set2 
test   train     2811
       valid      723
train  train    22001
       valid     5480
Name: sentiment, dtype: int64

In [73]:
data.groupby(["set", "set2"]).apply(lambda x : accuracy_score(y_true=x.sentiment, y_pred=x.predicted_sentiment))

set    set2 
test   train    0.899324
       valid    0.777317
train  train    0.891959
       valid    0.772810
dtype: float64

In [74]:
sentiment_lookup

{'positive': 2, 'neutral': 1, 'negative': 0}

In [75]:
data.loc[(data.sentiment == "neutral") & (data.predicted_sentiment == "negative")] # most incorrect in this cohort

,text,set,sentiment,predicted_sentiment,set2
0,i miss jack n box and whataburger and oooo taco cabanaaaaaa lmao,train,neutral,negative,train
64,Byebye,train,neutral,negative,train
81,Doggone it! I only got an hour nap in I really want some Ponderosa chicken wings!,train,neutral,negative,train
112,My radiator boiled out,train,neutral,negative,train
173,_precious06 sooo mad,train,neutral,negative,train
...,...,...,...,...,...
30723,LOL that is so Charlie......I miss Charlie,train,neutral,negative,train
30887,broke my gorilla pod and I think one Lego Knight lost his helmet,train,neutral,negative,train
30979,mama the boys gon be sweating her hard in Pre K lol muah,train,neutral,negative,train
30989,Sitting at my desk wishing things were different,train,neutral,negative,valid


### Span Detection Model Fit

In [76]:
for i in range(NUM_FOLDS):
    if os.path.exists(RESULTS_DIR+MODEL_PREFIX+"_LossLogs_"+str(i)+".csv"):
        os.remove(RESULTS_DIR+MODEL_PREFIX+"_LossLogs_"+str(i)+".csv")

In [77]:
def print_metrics(pred_dict):
    print("[INFO] ","="*15,"Validation for FOLD#", num, "="*15)
    funcs = [accuracy_score, f1_score, precision_score, recall_score, confusion_matrix]
    for f in funcs:
        for data_set in ["train","valid"]:
            for var in ["starts", "stops"]:
                if f in [accuracy_score]:
                    res = f(**pred_dict[data_set][var])
                    print("[INFO] {:.2f}".format(100 * res), "\t||", data_set, "\t||", var, "\t||", f.__name__)
                elif f in [confusion_matrix]:
                    res = f(**pred_dict[data_set][var], labels=np.arange(MAX_SEQ_LEN))
                    np.savetxt(X=res, fmt='%i', delimiter=",",
                               fname=RESULTS_DIR+"ConfusionMatrix_"+MODEL_PREFIX+"_"+data_set+"_"+var+".csv")
                    print("[INFO] \t||", data_set, "\t||", var, "\t||", f.__name__, "\t||", 
                          RESULTS_DIR+"ConfusionMatrix_"+MODEL_PREFIX+"_"+data_set+"_"+var+".csv")
                else:
                    res = f(**pred_dict[data_set][var], average="macro")
                    print("[INFO] {:.2f}".format(100 * res), "\t||", data_set, "\t||", var, "\t||", f.__name__)
        print("=======================================================================")

In [78]:
def post_process(string):
    string = re.sub(pattern=" (negative|positive|neutral)[ ]+$", repl="", string=string)
    string = re.sub(pattern=" +", repl=" ", string=string.strip())
    return string

<a href="https://keras.io/guides/transfer_learning/#finetuning" target="_blank"><h2 id="finetuning">Fine-tuning</h2></a>
<p>Once your model has converged on the new data, you can try to unfreeze all or part of
 the base model and retrain the whole model end-to-end with a very low learning rate.</p>
 <p>This is an optional last step that can potentially give you incremental improvements.
 It could also potentially lead to quick overfitting -- keep that in mind.</p>
 <p>It is critical to only do this step <em>after</em> the model with frozen layers has been
trained to convergence. If you mix randomly-initialized trainable layers with
trainable layers that hold pre-trained features, the randomly-initialized layers will
cause very large gradient updates during training, which will destroy your pre-trained
 features.</p>
 <p>It's also critical to use a very low learning rate at this stage, because
you are training a much larger model than in the first round of training, on a dataset
 that is typically very small.
As a result, you are at risk of overfitting very quickly if you apply large weight
 updates. Here, you only want to readapt the pretrained weights in an incremental way.</p>

<a href="https://keras.io/guides/transfer_learning/#finetuning" target="_blank"><p><strong>Important note about <code>compile()</code> and <code>trainable</code></strong></p></a>
<p>Calling <code>compile()</code> on a model is meant to "freeze" the behavior of that model. This
 implies that the <code>trainable</code>
attribute values at the time the model is compiled should be preserved throughout the
 lifetime of that model,
until <code>compile</code> is called again. Hence, if you change any <code>trainable</code> value, make sure
 to call <code>compile()</code> again on your
model for your changes to be taken into account.</p>

In [79]:
span_detection_model_bkup = span_detection_model

In [80]:
for num, (t_index, v_index) in enumerate(sss.split(X_span, Y_span_stops)): # NOTICE the swap of val and train indices, on purpose for faster training
    print("[INFO] ==================== FOLD#", num, "====================")
    
    if num > 0:
        del span_history
        del span_detection_model
        gc.collect()
        K.clear_session()
    
    span_detection_model = span_detection_model_bkup
    
    span_mcp = ModelCheckpoint(filepath=RESULTS_DIR+MODEL_PREFIX+"BestCheckpoint_"+str(num)+".h5", monitor='val_loss',
                               verbose=0, save_best_only=False, save_weights_only=True, mode='auto', save_freq='epoch')
    
    span_csvl = CSVLogger(filename=RESULTS_DIR+MODEL_PREFIX+"_LossLogs_"+str(num)+".csv",
                          separator=",",
                          append=True)
    
    print("[INFO] Training only the final layers at higher learning rates.")
    span_detection_model.layers[3].trainable = False
    adam = Adam(learning_rate=MAX_LR)
    span_detection_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                 optimizer=adam,
                                 metrics=['accuracy'],
                                 loss_weights={"starts_0":1.0,"stops_0":1.0,"starts_1":1.0,"stops_1":1.0})
    span_history = span_detection_model.fit(x={"att_flags":X_span_att[t_index],
                                               "words":X_span[t_index],
                                               "token_ids":np.zeros_like(X_span_att[t_index])},
                                            y={"starts_0":Y_span_starts[t_index],
                                               "stops_0":Y_span_stops[t_index], 
                                               "starts_1":Y_span_starts[t_index],
                                               "stops_1":Y_span_stops[t_index]},
                                            shuffle=True,
                                            batch_size=BATCH_SIZE,
                                            epochs=NUM_EPOCHS[0],
                                            validation_data=({"att_flags":X_span_att[v_index],
                                                              "words":X_span[v_index],
                                                              "token_ids":np.zeros_like(X_span_att[v_index])},
                                                             {"starts_0":Y_span_starts[v_index],
                                                              "stops_0":Y_span_stops[v_index], 
                                                              "starts_1":Y_span_starts[v_index],
                                                              "stops_1":Y_span_stops[v_index]}),
                                            verbose=1,
                                            callbacks=[span_mcp, span_csvl])
    
    print("[INFO] Training only the final layers at lower learning rates.")
    adam = Adam(learning_rate=MID_LR)
    span_detection_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                 optimizer=adam,
                                 metrics=['accuracy'],
                                 loss_weights={"starts_0":1.0,"stops_0":1.0,"starts_1":1.0,"stops_1":1.0})
    span_history = span_detection_model.fit(x={"att_flags":X_span_att[t_index],
                                               "words":X_span[t_index],
                                               "token_ids":np.zeros_like(X_span_att[t_index])},
                                            y={"starts_0":Y_span_starts[t_index],
                                               "stops_0":Y_span_stops[t_index], 
                                               "starts_1":Y_span_starts[t_index],
                                               "stops_1":Y_span_stops[t_index]},
                                            shuffle=True,
                                            batch_size=BATCH_SIZE,
                                            epochs=NUM_EPOCHS[1],
                                            validation_data=({"att_flags":X_span_att[v_index],
                                                              "words":X_span[v_index],
                                                              "token_ids":np.zeros_like(X_span_att[v_index])},
                                                             {"starts_0":Y_span_starts[v_index],
                                                              "stops_0":Y_span_stops[v_index], 
                                                              "starts_1":Y_span_starts[v_index],
                                                              "stops_1":Y_span_stops[v_index]}),
                                            verbose=1,
                                            callbacks=[span_mcp, span_csvl])
    
    print("[INFO] Unfreezing RoBerta layer and training at lowest learning rates.")
    span_detection_model.layers[3].trainable = True
    adam = Adam(learning_rate=MIN_LR)
    span_detection_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                 optimizer=adam,
                                 metrics=['accuracy'],
                                 loss_weights={"starts_0":1.0,"stops_0":1.0,"starts_1":1.0,"stops_1":1.0})
    span_history = span_detection_model.fit(x={"att_flags":X_span_att[t_index],
                                               "words":X_span[t_index],
                                               "token_ids":np.zeros_like(X_span_att[t_index])},
                                            y={"starts_0":Y_span_starts[t_index],
                                               "stops_0":Y_span_stops[t_index], 
                                               "starts_1":Y_span_starts[t_index],
                                               "stops_1":Y_span_stops[t_index]},
                                            shuffle=True,
                                            batch_size=BATCH_SIZE,
                                            epochs=NUM_EPOCHS[2],
                                            validation_data=({"att_flags":X_span_att[v_index],
                                                              "words":X_span[v_index],
                                                              "token_ids":np.zeros_like(X_span_att[v_index])},
                                                             {"starts_0":Y_span_starts[v_index],
                                                              "stops_0":Y_span_stops[v_index], 
                                                              "starts_1":Y_span_starts[v_index],
                                                              "stops_1":Y_span_stops[v_index]}),
                                            verbose=1,
                                            callbacks=[span_mcp, span_csvl])
    
    pred_train = span_detection_model.predict(x = {"att_flags":X_span_att[t_index],
                                                   "words":X_span[t_index],
                                                   "token_ids":np.zeros_like(X_span_att[t_index])},
                                              batch_size=PREDICT_BATCH_SIZE)

    pred_val = span_detection_model.predict(x = {"att_flags":X_span_att[v_index],
                                                 "words":X_span[v_index],
                                                 "token_ids":np.zeros_like(X_span_att[v_index])},
                                            batch_size=PREDICT_BATCH_SIZE)
    
    pred_starts_train, pred_stops_train = (pred_train[0]/2.0 + pred_train[2]/2.0), (pred_train[1]/2.0 + pred_train[3]/2.0)
    pred_starts_val, pred_stops_val = (pred_val[0]/2.0 + pred_val[2]/2.0), (pred_val[1]/2.0 + pred_val[3]/2.0)
    
    # Accumulate test results after training every fold
    pred_test_fold = span_detection_model.predict(x = {"att_flags":X_span_att_test,
                                                       "words":X_span_test,
                                                       "token_ids":np.zeros_like(X_span_att_test)},
                                                  batch_size=PREDICT_BATCH_SIZE)
    if num==0:
        pred_test = []
        pred_test.append(pred_test_fold[0]/2.0 + pred_test_fold[2]/2.0)
        pred_test.append(pred_test_fold[1]/2.0 + pred_test_fold[3]/2.0)
    else:
        pred_test[0] += (pred_test_fold[0]/2.0 + pred_test_fold[2]/2.0)
        pred_test[1] += (pred_test_fold[1]/2.0 + pred_test_fold[3]/2.0)
    
    # Tabulate
    preds = {
        "train":{
            "starts":{
                "y_true":Y_span_starts[t_index],
                "y_pred":pred_train[0].argmax(axis=1)
            },
            "stops":{
                "y_true":Y_span_stops[t_index],
                "y_pred":pred_train[1].argmax(axis=1)
            }
        },
        "valid":{
            "starts":{
                "y_true":Y_span_starts[v_index],
                "y_pred":pred_val[0].argmax(axis=1)
            },
            "stops":{
                "y_true":Y_span_stops[v_index],
                "y_pred":pred_val[1].argmax(axis=1)
            }        
        }
    }
    
    print_metrics(pred_dict=preds)

    print("[INFO] Prediction shape for training data: ", pred_starts_train.shape, pred_stops_train.shape)
    print("[INFO] Prediction shape for validation data: ", pred_starts_val.shape, pred_stops_val.shape)

    print("[INFO] Normal predictions (StartIndex less than EndIndex) for training data: ",
          sum([s<e for s,e in zip(pred_starts_train.argmax(axis=1),
                                  pred_stops_train.argmax(axis=1))]),
          "out of", pred_starts_train.shape[0])
    print("[INFO] Normal predictions (StartIndex less than EndIndex) for validation data: ",
          sum([s<e for s,e in zip(pred_starts_val.argmax(axis=1),
                                  pred_stops_val.argmax(axis=1))]),
          "out of", pred_starts_val.shape[0],)

    pred_words_train = [post_process(tokenizer.decode(t[s:e])) if s<e else post_process(tokenizer.decode(t[e:])) for t, s,e in zip(X_span[t_index],
                                                                                                                                   pred_starts_train.argmax(axis=1),
                                                                                                                                   pred_stops_train.argmax(axis=1))]
    
    pred_words_val = [post_process(tokenizer.decode(t[s:e])) if s<e else post_process(tokenizer.decode(t[e:])) for t, s,e in zip(X_span[v_index],
                                                                                                                                 pred_starts_val.argmax(axis=1),
                                                                                                                                 pred_stops_val.argmax(axis=1))]

    print("[INFO] Training Jaccard Score: ",
          np.mean([jaccard(str1=i, str2=j) for i,j in zip([t for n,t in enumerate(Y_span_words) if n in t_index],
                                                          pred_words_train)]))
    print("[INFO] Validation Jaccard Score: ",
          np.mean([jaccard(str1=i, str2=j) for i,j in zip([t for n,t in enumerate(Y_span_words) if n in v_index],
                                                          pred_words_val)]))
    print("[INFO] Training for fold:", num, "finished at", ctime(time()))
print(ctime(time()))

[INFO] ==================== FOLD# 0 ====================
[INFO] Training only the final layers at higher learning rates.
Train on 11212 samples, validate on 2803 samples
Epoch 1/5
11212/11212 [==============================] - 151s 13ms/sample - loss: 9.0147 - starts_0_loss: 1.7647 - stops_0_loss: 1.9347 - starts_1_loss: 2.4682 - stops_1_loss: 2.8467 - starts_0_accuracy: 0.4418 - stops_0_accuracy: 0.4243 - starts_1_accuracy: 0.3738 - stops_1_accuracy: 0.3357 - val_loss: 7.0804 - val_starts_0_loss: 1.3891 - val_stops_0_loss: 1.6999 - val_starts_1_loss: 1.8655 - val_stops_1_loss: 2.1227 - val_starts_0_accuracy: 0.5173 - val_stops_0_accuracy: 0.4720 - val_starts_1_accuracy: 0.4502 - val_stops_1_accuracy: 0.4866
Epoch 2/5
11212/11212 [==============================] - 133s 12ms/sample - loss: 7.5766 - starts_0_loss: 1.7399 - stops_0_loss: 1.8820 - starts_1_loss: 1.8452 - stops_1_loss: 2.1094 - starts_0_accuracy: 0.4663 - stops_0_accuracy: 0.4486 - starts_1_accuracy: 0.4552 - stops_1_accura

Epoch 5/5
11212/11212 [==============================] - 297s 27ms/sample - loss: 5.1270 - starts_0_loss: 1.1649 - stops_0_loss: 1.2033 - starts_1_loss: 1.3089 - stops_1_loss: 1.4502 - starts_0_accuracy: 0.5715 - stops_0_accuracy: 0.5710 - starts_1_accuracy: 0.5823 - stops_1_accuracy: 0.6013 - val_loss: 5.2125 - val_starts_0_loss: 1.1399 - val_stops_0_loss: 1.1555 - val_starts_1_loss: 1.3825 - val_stops_1_loss: 1.5304 - val_starts_0_accuracy: 0.5722 - val_stops_0_accuracy: 0.5669 - val_starts_1_accuracy: 0.5569 - val_stops_1_accuracy: 0.5837
[INFO]  =============== Validation for FOLD# 0 ===============
[INFO] 61.02 	|| train 	|| starts 	|| accuracy_score
[INFO] 60.16 	|| train 	|| stops 	|| accuracy_score
[INFO] 57.22 	|| valid 	|| starts 	|| accuracy_score
[INFO] 56.69 	|| valid 	|| stops 	|| accuracy_score
[INFO] 52.29 	|| train 	|| starts 	|| f1_score
[INFO] 51.12 	|| train 	|| stops 	|| f1_score
[INFO] 47.64 	|| valid 	|| starts 	|| f1_score
[INFO] 47.29 	|| valid 	|| stops 	|| f1

c:\users\deepak\miniconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\deepak\miniconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


 train 	|| starts 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_train_starts.csv
[INFO] 	|| train 	|| stops 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_train_stops.csv
[INFO] 	|| valid 	|| starts 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_valid_starts.csv
[INFO] 	|| valid 	|| stops 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_valid_stops.csv
[INFO] Prediction shape for training data:  (11212, 100) (11212, 100)
[INFO] Prediction shape for validation data:  (2803, 100) (2803, 100)
[INFO] Normal predictions (StartIndex less than EndIndex) for training data:  7973 out of 11212
[INFO] Normal predictions (StartIndex less than EndIndex) for validation data:  1965 out of 2803
[INFO] Training Jaccard Score:  0.5430792269610896
[INFO] Validation Jaccard Score:  0.526457102760759
[INFO] Training for fold: 0 finished at Mon Jun  1 03:30:22 2020
[INFO] ==================== FOLD# 1 ====================
[INFO] Training only the final layers at higher lear

Epoch 3/5
11212/11212 [==============================] - 298s 27ms/sample - loss: 4.7934 - starts_0_loss: 1.1005 - stops_0_loss: 1.1220 - starts_1_loss: 1.2166 - stops_1_loss: 1.3541 - starts_0_accuracy: 0.5912 - stops_0_accuracy: 0.5867 - starts_1_accuracy: 0.6029 - stops_1_accuracy: 0.6208 - val_loss: 4.6823 - val_starts_0_loss: 1.0295 - val_stops_0_loss: 1.0462 - val_starts_1_loss: 1.2218 - val_stops_1_loss: 1.3827 - val_starts_0_accuracy: 0.5865 - val_stops_0_accuracy: 0.6068 - val_starts_1_accuracy: 0.6047 - val_stops_1_accuracy: 0.6204
Epoch 4/5
11212/11212 [==============================] - 300s 27ms/sample - loss: 4.7131 - starts_0_loss: 1.0926 - stops_0_loss: 1.0917 - starts_1_loss: 1.2043 - stops_1_loss: 1.3243 - starts_0_accuracy: 0.5933 - stops_0_accuracy: 0.5927 - starts_1_accuracy: 0.6044 - stops_1_accuracy: 0.6268 - val_loss: 4.6771 - val_starts_0_loss: 1.0242 - val_stops_0_loss: 1.0505 - val_starts_1_loss: 1.2292 - val_stops_1_loss: 1.3712 - val_starts_0_accuracy: 0.587

...

c:\users\deepak\miniconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\deepak\miniconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[INFO] 	|| train 	|| starts 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_train_starts.csv
[INFO] 	|| train 	|| stops 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_train_stops.csv
[INFO] 	|| valid 	|| starts 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_valid_starts.csv
[INFO] 	|| valid 	|| stops 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_valid_stops.csv
[INFO] Prediction shape for training data:  (11212, 100) (11212, 100)
[INFO] Prediction shape for validation data:  (2803, 100) (2803, 100)
[INFO] Normal predictions (StartIndex less than EndIndex) for training data:  7891 out of 11212
[INFO] Normal predictions (StartIndex less than EndIndex) for validation data:  1970 out of 2803
[INFO] Training Jaccard Score:  0.5749323053356906
[INFO] Validation Jaccard Score:  0.564886924031743
[INFO] Training for fold: 3 finished at Mon Jun  1 06:00:18 2020
[INFO] ==================== FOLD# 4 ====================
[INFO] Training only the final layers at h

Epoch 3/5
11212/11212 [==============================] - 299s 27ms/sample - loss: 4.0146 - starts_0_loss: 0.9337 - stops_0_loss: 0.9285 - starts_1_loss: 1.0274 - stops_1_loss: 1.1251 - starts_0_accuracy: 0.6457 - stops_0_accuracy: 0.6559 - starts_1_accuracy: 0.6564 - stops_1_accuracy: 0.6841 - val_loss: 3.8139 - val_starts_0_loss: 0.8532 - val_stops_0_loss: 0.8656 - val_starts_1_loss: 1.0189 - val_stops_1_loss: 1.0805 - val_starts_0_accuracy: 0.6711 - val_stops_0_accuracy: 0.6789 - val_starts_1_accuracy: 0.6743 - val_stops_1_accuracy: 0.6982
Epoch 4/5
11212/11212 [==============================] - 720s 64ms/sample - loss: 3.9713 - starts_0_loss: 0.9260 - stops_0_loss: 0.9129 - starts_1_loss: 1.0224 - stops_1_loss: 1.1103 - starts_0_accuracy: 0.6448 - stops_0_accuracy: 0.6596 - starts_1_accuracy: 0.6636 - stops_1_accuracy: 0.6884 - val_loss: 3.8263 - val_starts_0_loss: 0.8536 - val_stops_0_loss: 0.8745 - val_starts_1_loss: 1.0213 - val_stops_1_loss: 1.0814 - val_starts_0_accuracy: 0.675

c:\users\deepak\miniconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\deepak\miniconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[INFO] 	|| train 	|| starts 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_train_starts.csv
[INFO] 	|| train 	|| stops 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_train_stops.csv
[INFO] 	|| valid 	|| starts 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_valid_starts.csv
[INFO] 	|| valid 	|| stops 	|| confusion_matrix 	|| ../results/ConfusionMatrix_V24_valid_stops.csv
[INFO] Prediction shape for training data:  (11212, 100) (11212, 100)
[INFO] Prediction shape for validation data:  (2803, 100) (2803, 100)
[INFO] Normal predictions (StartIndex less than EndIndex) for training data:  7322 out of 11212
[INFO] Normal predictions (StartIndex less than EndIndex) for validation data:  1860 out of 2803
[INFO] Training Jaccard Score:  0.5772569418058389
[INFO] Validation Jaccard Score:  0.5654481260498994
[INFO] Training for fold: 4 finished at Mon Jun  1 06:58:52 2020
Mon Jun  1 06:58:52 2020


## Validation

#### Inference

In [81]:
pred_starts_test, pred_stops_test = pred_test[0]/NUM_FOLDS, pred_test[1]/NUM_FOLDS
print("[INFO] Prediction shape for testing data: ", pred_starts_test.shape, pred_stops_test.shape)

[INFO] Prediction shape for testing data:  (3534, 100) (3534, 100)


#### Postprocessing

In [82]:
print("Normal predictions (StartIndex less than EndIndex) for testing data:",
      sum([s<e for s,e in zip(pred_starts_test.argmax(axis=1),
                              pred_stops_test.argmax(axis=1))]), 
      "out of",
      pred_starts_test.shape[0])

Normal predictions (StartIndex less than EndIndex) for testing data: 2764 out of 3534


In [83]:
pred_words_test = [
    post_process(tokenizer.decode(t[s:e+1])) if s<e else post_process(tokenizer.decode(t[e:])) for t,s,e in zip(X_span_test,
                                                                                                              pred_starts_test.argmax(axis=1),
                                                                                                              pred_stops_test.argmax(axis=1))
]

In [84]:
check_idx = 158
#print([[t,i,j,k] for t,i,j,k in zip(tokenizer.decode(),X_test[check_idx],pred_starts_test[check_idx],pred_stops_test[check_idx])])
print(tokenizer.decode(X_span_test[check_idx]))
print(pred_starts_test.argmax(axis=1)[check_idx])
print(pred_stops_test.argmax(axis=1)[check_idx])
print(post_process(tokenizer.decode(X_span_test[check_idx][pred_starts_test.argmax(axis=1)[check_idx]:1+pred_stops_test.argmax(axis=1)[check_idx]])))

 yes! im down to 50% full on my dvr  i was at 98% like 3 days ago... lol i swear if i didnt have a dvr i would never watch tv  neutral 
25
38
i swear if i didnt have a dvr i would never watch tv


## Submission

In [85]:
test_df_span['selected_text'] = pred_words_test

In [86]:
test_df_span["selected_text"] = np.where(test_df_span["sentiment"] == "neutral",
                                         test_df_span["text"],
                                         test_df_span["selected_text"])

In [87]:
test_df_span[["textID", "selected_text"]].to_csv(RESULTS_DIR+"submission_"+MODEL_PREFIX+".csv", index=False)

In [88]:
test_df_span.loc[test_df_span.sentiment!="neutral"][["text", "sentiment","selected_text"]].sample(25)

,text,sentiment,selected_text
724,This morning I rode behind a guy with a bird cage containing a plastic Tyrannosaurus rex attached to his bicycle. I couldn`t get a photo.,negative,i couldn`t get a photo.
1306,good luck in the pressure test! don`t worry everything will be great,positive,good luck in the pressure test! don`t worry everything will be great
528,"Perfect night. Best month of my life so far!! You my boo,",positive,perfect night.
2893,bye LA... I already miss you,negative,miss you
3442,"so did you watch? I`d forgottenmost of it, enjoyed re-seeing it, but paying for it today, my face doesnt bounce back anymore",positive,"enjoyed re-seeing it, but paying for it today, my face doesnt bounce back anymore"
288,congrats! Photo of dre?,positive,congrats!
1453,yes yes yes lotsa fun i cant wait. dont like make up with shaun too much in front of me tho cos ill get awkard haha,positive,fun i cant wait. dont like make up with shaun too much in front of me tho cos ill get awkard haha
2898,Happy Star Wars Day,positive,happy star wars day
207,crying real tears,negative,crying real tears
203,I hate my life,negative,i hate my life
